## Ucitavanje baze

In [15]:
from funkcije import *

In [16]:
baza_location = biraj_bazu()

FileChooser(path='/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav', filen…

In [17]:
baza = ucitaj_bazu(baza_location)

greska sa citanjem fajla:/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/baza/emodb/wav/newbaza


/home/zarko/PFE/prepoznavanje-emocija/lk-s-2023-prepoznavanje-emocija/funkcije.py:129: UserWarning: PySoundFile failed. Trying audioread instead.
  data, samplerate = librosa.load(fc.selected + filename)
/home/zarko/.local/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [18]:
#liste ficera 
hurstovi = [] #+
duzine = [] #+
odnostig = [] #+
najduzaprica = [] #+

zcrovi = [] #lokalni
pitchovi = []

zcr_prosek = [] #globalni #+
zcr_min = []
zcr_max = []
zcr_med = []
zcr_stdev = []

pitch_prosek = [] #globalni 
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

energije_ukupne = []
energije_short_term = []
energije_short_term_silence = []

eng_prosek = [] #globalni 
eng_min = []
eng_max = []
eng_med = []
eng_stdev = []

engbeztis_prosek = [] #globalni 
engbeztis_min = []
engbeztis_max = []
engbeztis_med = []
engbeztis_stdev = []

frame_lenght = 480 
hop_lenght = frame_lenght//2

mfccovi = []
mfccovi_array = []


frame_length_tisina = 100
hop_length_tisina = frame_length_tisina//2
granica = 0.01

---
## hurst 

In [7]:

for sample in baza:
        h, c, val = compute_Hc(sample.data)
        hurstovi.append(h)

### duzine

In [8]:


for sample in baza:

     duzina = len(sample.data) * 1/sample.samplerate
        
     duzine.append(duzina) 

In [9]:
### odnos govor prema tisina, max govor 

duzinatisine = []

#  frejmovano


for sample in baza:
      
     duzina = len(sample.data) * 1/sample.samplerate
     duzinatisinepriv = 0
     ptisina = 0
     trenprica = 0
     maxprica = 0
     for i in range (0, len(sample.data), hop_length_tisina):
          engparceta = sum(abs(sample.data[i:i+frame_length_tisina]**2))
          if engparceta < granica:
               trenprica = 0
               if ptisina == 0:
                    duzinatisinepriv = duzinatisinepriv + frame_length_tisina * 1/sample.samplerate
               else :
                    duzinatisinepriv = duzinatisinepriv + hop_length_tisina * 1/sample.samplerate
               ptisina = 1
          else: 
               ptisina = 0
               if trenprica == 0:
                    trenprica = frame_length_tisina * 1/sample.samplerate
               else :
                    trenprica = trenprica + hop_length_tisina * 1/sample.samplerate
               if trenprica > maxprica: maxprica = trenprica
     duzinatisine.append(duzinatisinepriv)
     najduzaprica.append(maxprica)

for i in range (0, len(duzine)):
     if duzinatisine[i] > 0:
          odnostig.append((duzine[i]-duzinatisine[i])/duzinatisine[i])
     else: odnostig.append(1)      

## MFCC
---

In [19]:
# data # citav audio fajl u nizu
# parcici[0] # prvi segment tog audio fajla(mozda nije prvi zbog tisine zapravo) --- NE
# prvi_seg # prvi segment koji nije tisina(trenutno su ostali pobrisani ovde)

mfccovi_array = []

for sample in baza:
    duzina = len(sample.data) * 1/sample.samplerate
            
    matrica = librosa.feature.mfcc (y = sample.data, sr = sample.samplerate)
    mfccovi.append(matrica) 
    niz = matrica.flatten()
    mfccovi_array.append(niz)


535


In [12]:
print()


#for i in range (0, len(mfccovi)):
 #   print(mfccovi[i])
  #  print(mfccovi_array[i])

## ZCR
---

In [13]:
## lokalni 


for sample in baza:
        #pitch = librosa.feature.zero_crossing_rate(data, frame_length = frame_length, hop_length = hop_length)
        zcr = librosa.feature.zero_crossing_rate(sample.data, frame_length = frame_lenght, hop_length = hop_lenght)
        #print(pitch)
        #print(zcr[0])         
        #plt.plot(zcr)      #vraca matricu iz nekog razloga, ali je sav sadrzaj u zcr[0] tkd cu samo to da uzimam
        zcrovi.append(zcr[0])

In [14]:
## globalni 

for zcr in zcrovi:
    zcr_prosek.append(np.mean(zcr))
    zcr_min.append(np.min(zcr))
    zcr_max.append(np.max(zcr))
    zcr_med.append(np.median(zcr))
    zcr_stdev.append(np.std(zcr))
    #print (prosek, min, max, med, stdev)
    #break

## F0 --- pitch
---

In [15]:
#lokalni

for sample in baza:

    f0, voiced_flag, voiced_probs = librosa.pyin(sample.data,

                                                fmin=librosa.note_to_hz('C2'),

                                                fmax=librosa.note_to_hz('C7'), sr=sample.samplerate) #, frame_length=frame_len, hop_length=hop_len)
    pitchovi.append(f0)

In [16]:
## globalni 

pitch_prosek = []
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

for pitch_nn in pitchovi:
    pitch = pitch_nn[pitch_nn > -1]
    pitch_prosek.append(np.mean(pitch))
    pitch_min.append(np.min(pitch))
    pitch_max.append(np.max(pitch))
    pitch_med.append(np.median(pitch))
    pitch_stdev.append(np.std(pitch))
    #print (prosek, min, max, med, stdev)
    #break

## energija
---

In [29]:
# energija


# short term energija, frejmovano
frame_length = frame_length_tisina # videcemo ali ovo ce ukloniti vise tisine tkd trebalo bi da utice 
hop_length = frame_length//2


for sample in baza:

    energy = sum(abs(sample.data**2))  ## ukupna energija fajla
    energije_ukupne.append(energy)

    energy_arr = np.array([
        sum(abs( sample.data[i:i+frame_length]**2))
        for i in range(0, len(sample.data), hop_length)])

    energije_short_term_silence.append(energy_arr)
    
for energija_kratko in energije_short_term_silence:
    energije_short_term.append(energija_kratko[energija_kratko > granica])

In [30]:


for eng in energije_short_term:
    eng_prosek.append(np.mean(eng))
    eng_min.append(np.min(eng))
    eng_max.append(np.max(eng))
    eng_med.append(np.median(eng))
    eng_stdev.append(np.std(eng))

for engbeztis in energije_short_term_silence:
    engbeztis_prosek.append(np.mean(engbeztis))
    engbeztis_min.append(np.min(engbeztis))
    engbeztis_max.append(np.max(engbeztis))
    engbeztis_med.append(np.median(engbeztis))
    engbeztis_stdev.append(np.std(engbeztis)) 



---
## normalizacija

In [22]:

hurstovi_norm = preprocessing.normalize([np.asarray(hurstovi)])
duzine_norm = preprocessing.normalize([np.asarray(duzine)])
odnostig_norm = preprocessing.normalize([np.asarray(odnostig)])
najduzaprica_norm = preprocessing.normalize([np.asarray(najduzaprica)])

zcr_prosek_norm = preprocessing.normalize([np.asarray(zcr_prosek)])
zcr_min_norm = preprocessing.normalize([np.asarray(zcr_min)])
zcr_max_norm = preprocessing.normalize([np.asarray(zcr_max)])
zcr_med_norm = preprocessing.normalize([np.asarray(zcr_med)])
zcr_stdev_norm = preprocessing.normalize([np.asarray(zcr_stdev)])

pitch_prosek_norm = preprocessing.normalize([np.asarray(pitch_prosek)])
pitch_min_norm = preprocessing.normalize([np.asarray(pitch_min)])
pitch_max_norm = preprocessing.normalize([np.asarray(pitch_max)])
pitch_med_norm = preprocessing.normalize([np.asarray(pitch_med)])
pitch_stdev_norm = preprocessing.normalize([np.asarray(pitch_stdev)])

energije_ukupne_norm = preprocessing.normalize([np.asarray(energije_ukupne)])

eng_prosek_norm = preprocessing.normalize([np.asarray(eng_prosek)])
eng_min_norm = preprocessing.normalize([np.asarray(eng_min)])
eng_max_norm = preprocessing.normalize([np.asarray(eng_max)])
eng_med_norm = preprocessing.normalize([np.asarray(eng_med)])
eng_stdev_norm = preprocessing.normalize([np.asarray(eng_stdev)])

engbeztis_prosek_norm = preprocessing.normalize([np.asarray(engbeztis_prosek)])
engbeztis_min_norm = preprocessing.normalize([np.asarray(engbeztis_min)])
engbeztis_max_norm = preprocessing.normalize([np.asarray(engbeztis_max)])
engbeztis_med_norm = preprocessing.normalize([np.asarray(engbeztis_med)])
engbeztis_stdev_norm = preprocessing.normalize([np.asarray(engbeztis_stdev)])


In [31]:
energije_ukupne_norm = preprocessing.normalize([np.asarray(energije_ukupne)])

---
## ucitavanje ficera u fajlove 

### ficeri_glb

In [ ]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "w")
    f.close()

hurstovi;
duzine;
odnostig;
najduzaprica;
zcr_prosek;
zcr_min;
zcr_max;
zcr_med;
zcr_stdev;
pitch_prosek;
pitch_min;
pitch_max;
pitch_med;
pitch_stdev;

filename, govornik, recenica, emocija, emo, pol, godiste

In [ ]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb/" + name + ".txt", "a")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi_norm[i]) + ",")

    f.write(str(duzine_norm[i]) + ",")
    f.write(str(odnostig_norm[i]) + ",")
    f.write(str(najduzaprica_norm[i]) + ",")

    f.write(str(zcr_prosek_norm[i]) + ",")
    f.write(str(zcr_min_norm[i]) + ",")
    f.write(str(zcr_max_norm[i]) + ",")
    f.write(str(zcr_med_norm[i]) + ",")
    f.write(str(zcr_stdev_norm[i]) + ",")

    f.write(str(pitch_prosek_norm[i]) + ",")
    f.write(str(pitch_min_norm[i]) + ",")
    f.write(str(pitch_max_norm[i]) + ",")
    f.write(str(pitch_med_norm[i]) + ",")
    f.write(str(pitch_stdev_norm[i]) + ",")
    
    f.close()
    i = i+1

### ficeri_glb_norm

In [20]:


for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb_norm/" + name + ".txt", "w")
    f.close()

In [28]:

i = 0
for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_glb_norm/" + name + ".txt", "w")

    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")


    f.write(str(hurstovi_norm[0][i]) + ",")

    f.write(str(duzine_norm[0][i]) + ",")
    f.write(str(odnostig_norm[0][i]) + ",")
    f.write(str(najduzaprica_norm[0][i]) + ",")

    f.write(str(zcr_prosek_norm[0][i]) + ",")
    f.write(str(zcr_min_norm[0][i]) + ",")
    f.write(str(zcr_max_norm[0][i]) + ",")
    f.write(str(zcr_med_norm[0][i]) + ",")
    f.write(str(zcr_stdev_norm[0][i]) + ",")

    f.write(str(pitch_prosek_norm[0][i]) + ",")
    f.write(str(pitch_min_norm[0][i]) + ",")
    f.write(str(pitch_max_norm[0][i]) + ",")
    f.write(str(pitch_med_norm[0][i]) + ",")
    f.write(str(pitch_stdev_norm[0][i]) + ",")

    f.write(str(eng_prosek_norm[0][i]) + ",")
    f.write(str(eng_min_norm[0][i]) + ",")
    f.write(str(eng_max_norm[0][i]) + ",")
    f.write(str(eng_med_norm[0][i]) + ",")
    f.write(str(eng_stdev_norm[0][i]) + ",")

    f.write(str(engbeztis_prosek_norm[0][i]) + ",")
    f.write(str(engbeztis_min_norm[0][i]) + ",")
    f.write(str(engbeztis_max_norm[0][i]) + ",")
    f.write(str(engbeztis_med_norm[0][i]) + ",")
    f.write(str(engbeztis_stdev_norm[0][i]) + ",")
    
    f.close()
    i = i+1

In [ ]:

i = 0
f = open ("baza_norm.with_mfcccsv", "w")
for sample in baza:
    
    f.write(sample.filename + ",")
    f.write(sample.govornik + ",")
    f.write(sample.recenica + ",")
    f.write(sample.emo + ",")
    f.write(sample.emocija + ",")
    f.write(sample.pol + ",")
    f.write(sample.godiste + ",")

    f.write(str(hurstovi_norm[0][i]) + ",")

    f.write(str(duzine_norm[0][i]) + ",")
    f.write(str(odnostig_norm[0][i]) + ",")
    f.write(str(najduzaprica_norm[0][i]) + ",")

    f.write(str(zcr_prosek_norm[0][i]) + ",")
    f.write(str(zcr_min_norm[0][i]) + ",")
    f.write(str(zcr_max_norm[0][i]) + ",")
    f.write(str(zcr_med_norm[0][i]) + ",")
    f.write(str(zcr_stdev_norm[0][i]) + ",")

    f.write(str(pitch_prosek_norm[0][i]) + ",")
    f.write(str(pitch_min_norm[0][i]) + ",")
    f.write(str(pitch_max_norm[0][i]) + ",")
    f.write(str(pitch_med_norm[0][i]) + ",")
    f.write(str(pitch_stdev_norm[0][i]) + ",")

    f.write(str(eng_prosek_norm[0][i]) + ",")
    f.write(str(eng_min_norm[0][i]) + ",")
    f.write(str(eng_max_norm[0][i]) + ",")
    f.write(str(eng_med_norm[0][i]) + ",")
    f.write(str(eng_stdev_norm[0][i]) + ",")

    f.write(str(engbeztis_prosek_norm[0][i]) + ",")
    f.write(str(engbeztis_min_norm[0][i]) + ",")
    f.write(str(engbeztis_max_norm[0][i]) + ",")
    f.write(str(engbeztis_med_norm[0][i]) + ",")
    f.write(str(engbeztis_stdev_norm[0][i]))


    velicina = str(mfccovi[i].shape)[1:3] + "," + str(mfccovi[i].shape)[5:7]
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
        f.write("\n")
    
    i = i+1

f.close()_

In [47]:

i = 0
f = open ("baza_norm_with_mfcc.csv", "w")
for sample in baza:
    
    f.write(str(energije_ukupne_norm[0][i]))

    velicina = "," + str(mfccovi[i].shape[0]) + "," + str(mfccovi[i].shape[1])
    # print(velicina)
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
    f.write("\n")
    
    i = i+1

f.close()

### ficeri_mfcc

In [ ]:

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    f.close()

In [1]:
i = 0

for sample in baza:
    name = sample.filename[:7]
    f = open ("ficeri_mfcc/" + name + ".txt", "w")
    velicina = str(mfccovi[i].shape[0]) + "," + str(mfccovi[i].shape[1])
    f.write(velicina + ",")
    trenniz = mfccovi_array[i]
    for j in range (0, len(trenniz)):
        f.write(str(trenniz[j]) + ";")
    f.close()
    i=i+1

NameError: name 'baza' is not defined

In [ ]:
print(velicina)
#print(velicina)
#print(velicina[:1] + "," + velicina[1:] + ",")

In [ ]:
print()

## Ucitavanje svega nazad u nizove
---

In [38]:
df = pd.read_csv("baza_full.csv")

In [14]:
niz_svega = df.to_numpy()
for i in range(len(niz_svega)):
    hurstovi.append(niz_svega[i][7])
    duzine.append(niz_svega[i][8])
    
    odnostig.append(niz_svega[i][9])
    najduzaprica.append(niz_svega[i][10])
    zcr_prosek.append(niz_svega[i][11])
    zcr_min.append(niz_svega[i][12])
    zcr_max.append(niz_svega[i][13])
    zcr_med.append(niz_svega[i][14])
    zcr_stdev.append(niz_svega[i][15])
    
    pitch_prosek.append(niz_svega[i][16])
    pitch_min.append(niz_svega[i][17])
    pitch_max.append(niz_svega[i][18])
    pitch_med.append(niz_svega[i][19])
    pitch_stdev.append(niz_svega[i][20])

    energije_.append(niz_svega[i][21])
    ene
    

array([0.0515442507633738, 0.0235077763395934, 0.0120982921186082,
       0.0138193122344802, 0.0275233956105626, 0.0, 0.034284148903647,
       0.0225682281045526, 0.0325655103604418, 0.0250727454067386,
       0.0278902021831376, 0.0224684926455197, 0.0270782031045898,
       0.0269866557637608, 0.0426815529661139, 0.0420507357628377,
       0.0434511389230427, 0.0360259390003959, 0.0429544991929847,
       0.0339538874446746, 0.0005788916869813, 0.0434511389230427,
       0.0093490176060378, 0.0401642702829724, nan], dtype=object)

In [ ]:
# majko mila
#liste ficera 
hurstovi = [] #+
duzine = [] #+
odnostig = [] #+
najduzaprica = [] #+

# mfccovi = []

# zcrovi = [] #lokalni
# pitchovi = []

zcr_prosek = [] #globalni #+
zcr_min = []
zcr_max = []
zcr_med = []
zcr_stdev = []

pitch_prosek = [] #globalni 
pitch_min = []
pitch_max = []
pitch_med = []
pitch_stdev = []

energije_ukupne = []
energije_short_term = []
energije_short_term_silence = []

eng_prosek = [] #globalni 
eng_min = []
eng_max = []
eng_med = []
eng_stdev = []

engbeztis_prosek = [] #globalni 
engbeztis_min = []
engbeztis_max = []
engbeztis_med = []
engbeztis_stdev = []


In [48]:
df = pd.read_csv("baza_full.csv")

In [49]:
mfccovi[0].shape[0]

20

In [50]:
df

,filename,kod govornika,kod recenice,kod emocije,naziv emocije,pol,godiste,hurstovi,duzine,odnos tišine i govora,najduža priča,zcr prosek,zcr mean,zcr max,zcr med,zcr stdev,pitch prosek,pitch mean,pitch max,pitch med,pitch stdev,energija prosek,energija min,energija max,energija stdev,eng bez tis,eng bez tis avg,eng bez tis min,eng bez tis max,eng bez tis med,eng bez tis stdev,energija ukupna,mfccovi x,mfccovi y,mfccovi
0,03a01Fa.wav,3,a01,F,Sreća,M,31,0.057822,0.027695,0.014496,0.016678,0.039627,0.000000,0.041737,0.035844,0.040875,0.034708,0.031669,0.035054,0.034017,0.042673,0.017427,0.044139,0.032203,0.011907,0.025037,0.014647,0.002583,0.032203,0.007222,0.023040,0.009818,20,82,-573.01196;-582.12134;-586.8176;-584.4865;-338...
1,03a01Nc.wav,3,a01,N,Neutralno,M,31,0.051544,0.023508,0.012098,0.013819,0.027523,0.000000,0.034284,0.022568,0.032566,0.025073,0.027890,0.022468,0.027078,0.026987,0.042682,0.042051,0.043451,0.036026,0.042954,0.033954,0.000579,0.043451,0.009349,0.040164,0.019309,20,70,-544.9087;-524.06104;-542.695;-466.05853;-281....
2,03a01Wa.wav,3,a01,W,Bes,M,31,0.051150,0.027396,0.014676,0.016917,0.050864,0.100504,0.038425,0.063722,0.038340,0.039823,0.047725,0.032331,0.041398,0.028820,0.041732,0.042269,0.045070,0.024851,0.044305,0.035076,0.000232,0.045070,0.017133,0.041947,0.023258,20,81,-545.2024;-542.0179;-541.4934;-541.7991;-532.3...
3,03a02Fc.wav,3,a02,F,Sreća,M,31,0.048770,0.029270,0.015113,0.033119,0.042765,0.000000,0.046209,0.043809,0.040676,0.049342,0.045570,0.049004,0.046737,0.051125,0.039837,0.048514,0.055141,0.027702,0.043035,0.033512,0.002959,0.055141,0.015307,0.040674,0.023722,20,87,-462.67767;-450.47595;-474.7913;-311.72556;-17...
4,03a02Nc.wav,3,a02,N,Neutralno,M,31,0.047226,0.021006,0.019172,0.027877,0.027608,0.000000,0.036437,0.023896,0.032225,0.023681,0.028542,0.019003,0.025484,0.018476,0.057703,0.042335,0.052075,0.034992,0.058653,0.052259,0.007179,0.052075,0.018944,0.057258,0.026543,20,63,-544.0373;-531.3774;-527.38007;-405.45212;-268...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,16b10Lb.wav,16,b10,L,Dosada,F,31,0.049579,0.050227,0.034127,0.047772,0.037308,0.075378,0.043890,0.033189,0.039550,0.036696,0.042519,0.040970,0.032669,0.040968,0.034995,0.044069,0.039598,0.016916,0.040155,0.035540,0.002804,0.039598,0.017168,0.040335,0.043180,20,149,-463.54443;-465.73322;-494.0283;-500.79193;-46...
531,16b10Tb.wav,16,b10,T,Tuga,F,31,0.037642,0.051072,0.018778,0.060281,0.038834,0.000000,0.051343,0.029206,0.044661,0.041108,0.057747,0.038671,0.040220,0.017127,0.078883,0.042005,0.044063,0.058161,0.069282,0.071198,0.047161,0.044063,0.032171,0.068801,0.087927,20,151,-445.80164;-435.16284;-438.70038;-427.3457;-19...
532,16b10Td.wav,16,b10,T,Tuga,F,31,0.048029,0.057398,0.016357,0.058732,0.030288,0.000000,0.047203,0.018586,0.039758,0.033811,0.047177,0.026413,0.033626,0.012115,0.035342,0.042027,0.032165,0.033624,0.035049,0.030735,0.038350,0.032165,0.015337,0.033847,0.042653,20,170,-444.67496;-447.99628;-452.18787;-215.87308;-1...
533,16b10Wa.wav,16,b10,W,Bes,F,31,0.031072,0.035221,0.013295,0.042292,0.062608,0.175882,0.038756,0.086290,0.035835,0.058613,0.046367,0.063184,0.060262,0.071997,0.008845,0.042101,0.014207,0.003565,0.013078,0.007176,0.000096,0.014207,0.002408,0.011827,0.006113,20,104,-613.46625;-595.7472;-595.0384;-627.9332;-568....
